<a href="https://colab.research.google.com/github/syedahafsa12/Langgraph_Projects/blob/main/Gemini2_0_Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Study Assistant Chatbot with Gemini 2.0**

## **Features Implemented**
1. **Gemini 2.0**
   - Generates intelligent and context-aware responses using the live API.
   - Example: Responds with detailed guidance on topics like graphs.

2. **Live API**
   - Real-time response streaming for dynamic user interactions.

3. **LangGraph (Simulated)**
   - Decomposes user queries into subtasks for better understanding:
     - Identify graph type.
     - Understand difficulty level.
     - Suggest solutions.

4. **Text-Based Chat**
   - User interactions are managed through text.

5. **Human-in-the-Loop (HITL)**
   - Pauses to ask for more user input or refinement for better-tailored responses.

6. **Commands**
   - Supports commands like:
     - `help` - Lists available commands.
     - `exit` - Ends the chat session.

---


In [ ]:
!pip install -U -q google-genai
!pip install -q nest_asyncio


In [ ]:
from google.colab import userdata
from google import genai
import asyncio
import nest_asyncio

# Apply nested event loop for Colab
nest_asyncio.apply()

# Get API key
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Initialize the Gemini client
client = genai.Client(api_key=GOOGLE_API_KEY, http_options={'api_version': 'v1alpha'})

# Define the model
MODEL = "gemini-2.0-flash-exp"

async def decompose_query(query):
    """Mock function for decomposing study queries into subtasks."""
    if "plan" in query.lower():
        return ["Create a study schedule for efficient learning."]
    elif "summarize" in query.lower():
        return [f"Summarize the topic: {query.split('summarize', 1)[1].strip()}"]
    elif "explain" in query.lower():
        return [f"Explain the concept: {query.split('explain', 1)[1].strip()}"]
    else:
        return [query]

async def study_chatbot():
    print("Study Assistant Chatbot: Type 'exit' to quit or 'help' for commands.")

    config = {
        "generation_config": {"response_modalities": ["TEXT"]}
    }

    async with client.aio.live.connect(model=MODEL, config=config) as session:
        while True:
            user_input = input("\nYou: ")

            # Exit condition
            if user_input.lower() == "exit":
                print("Chat ended.")
                break

            # Help command
            if user_input.lower() == "help":
                print("\nCommands:\n1. 'summarize <topic>' - Summarize a topic\n"
                      "2. 'explain <concept>' - Explain a concept\n"
                      "3. 'plan study' - Get a study plan\n"
                      "4. 'review' - Interrupt and review response\n"
                      "5. 'exit' - Exit the chat")
                continue

            # Interrupt (Human-in-the-Loop)
            if user_input.lower() == "review":
                print("\n[HITL] Human intervention required. Review the response.")
                continue

            # Decompose query using LangGraph
            tasks = await decompose_query(user_input)

            # Process each subtask
            for task in tasks:
                prompt = f"As a study assistant, {task}"
                await session.send(prompt, end_of_turn=True)

                print("\nStudy Assistant:", end=" ")
                turn = session.receive()
                async for chunk in turn:
                    if chunk.text:
                        print(chunk.text, end="")
                print("\n")  # Line break for readability
await study_chatbot()



Study Assistant Chatbot: Type 'exit' to quit or 'help' for commands.

Study Assistant: Okay, I can definitely help you understand graphs! To give you the best assistance, let's break down this topic.  Tell me a little more about what you need. For example, it would be helpful if you can share some information such as:

**1. What kind of graphs are you studying?**

   *  **Math/Calculus Graphs:**  Are you focusing on function graphs (lines, parabolas, cubics, exponentials, trig functions, etc.), or coordinate systems, or are you working with specific concepts like limits, derivatives, and integrals that involve graphical interpretations?
   * **Statistics/Data Graphs:**  Are you dealing with bar graphs, histograms, pie charts, line graphs, scatter plots, box plots, or other types of data visualizations?
   * **Computer Science/Graph Theory Graphs:** Are you studying graphs as a way to represent relationships and connections between nodes (vertices) and edges? (e.g., directed graphs, und